In [4]:
import pandas as pd
import os
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Border, Side

# Replace 'file_path.xlsx' with the path to your Excel file
df = pd.read_excel("F:\Daily Reports\center_size.xlsx")

# Exclude rows with specific values in the 'app_status' column
filtered_df = df[~df['app_status'].isin(['REJECTED', 'DISBURSED', 'CB_VERIFICATION_FAILED','FOIR_REJECTED','CREDIT_SUBMIT_REJECTED','CLOSED','CBC_ERROR'])]

# Replace blank values in 'app_status' column with a placeholder
filtered_df['app_status'].fillna('Blank', inplace=True)

# Group the filtered DataFrame by the 'Area Name' column
grouped = filtered_df.groupby('Area name')

# Specify the base directory where you want to save the Excel files
base_directory = "F:\Area"

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

# Create a folder with today's date
today_directory = os.path.join(base_directory, today_date)
if not os.path.exists(today_directory):
    os.makedirs(today_directory)

# Iterate over the groups and save each group as a separate Excel file inside the today's date folder
for area, data in grouped:
    # Create the file path for each Excel file inside the today's date folder
    file_count = len(data)
    file_name = os.path.join(today_directory, f'{area}_{file_count}_{today_date}.xlsx')
    
    # Create a Pandas Excel writer for each area-wise file
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        # Write data to the first sheet
        data.to_excel(writer, sheet_name='Data', index=False)
        
        # Create pivot table in another sheet
        pivot_table_df = data.pivot_table(columns='app_status', index='Center Status', values='loan_id', aggfunc='count', fill_value=0, margins=True, margins_name='Grand Total')
        pivot_table_df.to_excel(writer, sheet_name='Pivot_Table')
        
        # Load the workbook
        workbook = writer.book
        worksheet = workbook['Pivot_Table']
        
        # Add borders to all cells in the pivot table
        for row in worksheet.iter_rows():
            for cell in row:
                cell.border = Border(left=Side(style='thin'), 
                                     right=Side(style='thin'), 
                                     top=Side(style='thin'), 
                                     bottom=Side(style='thin'))
        
        # Set column widths to justify content
        for column in worksheet.columns:
            max_length = 0
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column[0].column_letter].width = adjusted_width


C:\Users\Shubham Thakkar\AppData\Local\Temp\ipykernel_17536\448638919.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['app_status'].fillna('Blank', inplace=True)
